In [194]:
import numpy as np
import utils.optlib as opt
import plotly.graph_objects as go
import numba as nb

In [195]:
# setting the underlying
stock = opt.Underlying(price=65, volatility=0.3, interest=0.1)
stock_ = opt.Underlying(price=65, volatility=0.303, interest=0.1)

# creating the European Put option
EuPut = opt.Option(underlying=stock, style="European", call=False, strike=50)
EuPut_ = opt.Option(underlying=stock_, style="European", call=False, strike=50)

# creating the grid and adding option's parameters
grid0 = opt.Grid(xSteps=300, tSteps=200, xLeft=-3.0, xRight=3.0)
grid0.addOption(EuPut)
grid0.setBoundsPut()

grid_ = opt.Grid(xSteps=300, tSteps=200, xLeft=-3.0, xRight=3.0)
grid_.addOption(EuPut_)
grid_.setBoundsPut()

In [196]:
# creating matrices for schemes
@nb.njit()
def get_matricies(n, c):
    A = np.diag(np.ones(n+1) * (1 + c))
    B = np.diag(np.ones(n+1) * (1 - c))
    for i in range(1, n + 1):
        A[i, i-1] = -c/2
        A[i-1, i] = -c/2
        B[i, i-1] = c/2
        B[i-1, i] = c/2
    A[0,0] = 1
    A[n,n] = 1
    A[0,1] = 0
    A[n,n-1] = 0
    B[0,0] = 1
    B[n,n] = 1
    B[0,1] = 0
    B[n,n-1] = 0
    return A, B

# crank-nickolson scheme
def cn_scheme(grid: opt.Grid):
    A, B = get_matricies(grid0.xSteps, grid0.lamda)

    net = grid.net.copy()
    for i in np.arange(0, grid.tSteps):
        # explicit step
        f = np.dot(B, net[:, i].copy())
        # implicit step
        solution = opt.scalar_walk(A, f)
        net[:, i + 1] = solution

    grid.net = net.copy()
    return grid

### Vega with Finite Differencies

In [197]:
grid0 = cn_scheme(grid0)
grid_ = cn_scheme(grid_)
net0 = grid0.net
net_ = grid_.net
vega = (net0[250:,:] - net_[250:,:]) / (stock.volatility - stock_.volatility)

grid_.net = vega
grid_.plot(0, cut=False)


### Forward $y(x, \tau)$ and $dy/d\sigma$ computation

In [198]:
# for Vega
A, B = get_matricies(grid0.xSteps, grid0.lamda)

N = grid0.tSteps
M = grid0.xSteps
A_inv = np.linalg.inv(A)
Vega = np.empty((M+1, N+1))

def Y_sigma(time_ind, x_array):
    x = x_array[0]
    t = grid0.tGrid[time_ind]
    coef = -2 * stock.interest / stock.volatility**3

    y_sigma = coef * np.multiply(net0[:,0],  x_array)
    y_sigma[0] = coef * opt.g_func(grid0.q, t, x) * (x + (grid0.q + 1) * t)
    return y_sigma

lambda_sigma = stock.volatility * grid0.option.maturity * (M+1)**2 / ((N+1) * (grid0.xRight - grid0.xLeft)**2)
A_sigma = np.diag(2 * np.ones(M+1))
for i in range(1, M + 1):
    A[i, i-1] = -1
    A[i-1, i] = -1
A_sigma = A_sigma * lambda_sigma / 2
A_sigma[0,0] = 0
A_sigma[M,M] = 0
A_sigma[0,1] = 0
A_sigma[M,M-1] = 0

B_sigma = -A_sigma

In [199]:
# fwd vega calculation
Vega[:,0] = Y_sigma(0, grid0.xGrid.copy())

for i in np.arange(0, grid0.tSteps):
    Vega[:,i+1] = np.dot(A_inv, B_sigma @ net0[:,i] - A_sigma @ net0[:,i+1] + B @ Vega[:,i])

In [200]:
vega_grid = opt.Grid(xSteps=300,
                tSteps=200,
                xLeft=-3.0,
                xRight=3.0)

vega_grid.addOption(EuPut)
vega_grid.net = Vega[:250,:]
# vega_grid.toNormal(obj_mutation=True)
vega_grid.plot(0, cut=False, border=0.05)

### Adjoint $\lambda^{T}$ computation (backward loop)

In [201]:
N = grid.tSteps
M = grid.xSteps

lambda_vec = np.empty((M+1, N+1))
lambda_vec[:,N] = np.ones(M+1)

A_inv = np.linalg.inv(A)
f_y = np.dot(A_inv, B)

for i in range(N, 1, -1):
    lambda_vec[:, i-1] = np.dot(f_y, lambda_vec[:,i])

NameError: name 'grid' is not defined

### Adjoint Vega calculation (forward loop)

In [ ]:
lambda_sigma = stock.volatility * grid.option.maturity * (M+1)**2 / ((N+1) * (grid.xRight - grid.xLeft)**2)
A_sigma = np.diag(2 * np.ones(M+1))
for i in range(1, M + 1):
    A[i, i-1] = -1
    A[i-1, i] = -1
A_sigma = A_sigma * lambda_sigma / 2
B_sigma = -A_sigma

In [ ]:
Vega = np.empty((M+1, N+1))

x_vec =  grid.xGrid.copy()
y0_sigma = np.multiply(net[:,0], -2 * x_vec * stock.interest / stock.volatility**3)
Vega[:,0] = np.multiply(lambda_vec[:,0], y0_sigma)

def d_sigma(ind):
    x = grid.xGrid[0]
    t1 = grid.tGrid[ind]
    t2 = grid.tGrid[ind+1]
    q = grid.q

    el1 = opt.g_func(q, t1, x) + opt.g_func(q, t2, x)
    el2 = t1 * opt.g_func(q, t1, x) + t2 * opt.g_func(q, t2, x)

    d1 = el1 * lambda_sigma / 2
    d2 = (- 2 * stock.interest / stock.volatility**3) * (el1 * x + (q+1) * el2)
    return d1 + d2

# Adjoint vector of linear system
lambda_linear = np.dot(A_inv, np.ones(M+1))
for i in range(1, N+1):
    # linear system adjoint solving (finding foo_sigma)
    d_sigma_vec = np.zeros(M+1)
    d_sigma_vec[0] = d_sigma(i-1)
    something = B_sigma @ net[:,i-1] + B @ Vega[:,i-1] + d_sigma_vec - A_sigma @ net[:,i]
    foo_sigma = np.multiply(lambda_linear, something)

    Vega[:,i] = Vega[:,i-1] + np.multiply(lambda_vec[:,i], foo_sigma)

In [ ]:
grid.net = Vega.copy()
grid.plot(0, cut=True)